In [1]:
import os
from time import sleep
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
cwd = os.getcwd()
BASE_DIR = os.path.dirname(cwd)
GECKODRIVER_DIR = os.path.join(BASE_DIR, '.geckodriver-firefox')
GECKODRIVER_PATH = os.path.join(GECKODRIVER_DIR, 'geckodriver')

In [3]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Firefox(executable_path=GECKODRIVER_PATH, options=options)

In [4]:
categories = [
    'https://www.amazon.in/gp/bestsellers/gift-cards/',
    'https://www.amazon.in/gp/bestsellers/videogames/',
    'https://www.amazon.in/gp/bestsellers/electronics/',
]

In [5]:
first_url = categories[1]

In [6]:
driver.get(first_url)

In [7]:
body_element = driver.find_element_by_css_selector('body')
html_txt = body_element.get_attribute('innerHTML')

In [8]:
html_obj = HTML(html=html_txt)

In [9]:
new_links = [link for link in html_obj.links if link.startswith('/')]

# cleaned up links
new_links = [link for link in new_links if "product-reviews/" not in link]

In [10]:
page_links = [f"https://www.amazon.in{x}" for x in new_links]

In [11]:
print(page_links)

['https://www.amazon.in/Rs-3000-Sony-PlayStation-Network-Delivery/dp/B07K6PVR8B/ref=zg_bs_videogames_37/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34XVK67CGGA02', 'https://www.amazon.in/gcx/-/gfhz/?ref_=nav_cs_giftfinder_58f3a96b9747402884d8f629e69ebd28', 'https://www.amazon.in/pc-games/b/?ie=UTF8&node=1376518031&ref_=sv_v_5', 'https://www.amazon.in/health-and-personal-care/b/?ie=UTF8&node=1350384031&ref_=nav_cs_hpc_a024fc7ba9174802831aed18bbebb33d', 'https://www.amazon.in/business?ref=footer_aingw', 'https://www.amazon.in/music/prime?ref=footer_apm', 'https://www.amazon.in/gp/browse.html?node=3704992031&ref_=nav_cs_hi_b183765d90844c95b998beab0377167a', 'https://www.amazon.in/b?ie=UTF8&node=8872558031&ref_=footer_cares', 'https://www.amazon.in/gp/help/customer/display.html?ie=UTF8&nodeId=GDFU3JS5AL6SYHRD&ref_=footer_covid', 'https://www.amazon.in/WAR-HAMMER-GX1050-Gaming-Mousepad/dp/B07XTPKPTH/ref=zg_bs_videogames_17/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34

In [12]:
# simplifying lookup
def scrape_product_page(url, title_lookup = '#productTitle', price_lookup = '#priceblock_ourprice'):
    driver.get(url)
    sleep(1)
    body_element = driver.find_element_by_css_selector('body')
    html_txt = body_element.get_attribute('innerHTML')
    html_obj = HTML(html=html_txt)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [13]:
first_product_link = page_links[0]
print(first_product_link)

https://www.amazon.in/Rs-3000-Sony-PlayStation-Network-Delivery/dp/B07K6PVR8B/ref=zg_bs_videogames_37/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34XVK67CGGA02


In [14]:
for link in page_links:
    title, price = (None, None)
    try:
        title, price = scrape_product_page(url=link)
    except:
        pass
    
    if title is not None and price is not None:
        print('Title: ', title)
        print('Price: ', price)
        print('link: ', link, '\n')

Title:  Red Dead Redemption - 2 (PS4)
Price:  ₹ 2,230.00
link:  https://www.amazon.in/Red-Dead-Redemption-2-PS4/dp/B07BFGGNT9/ref=zg_bs_videogames_29/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34XVK67CGGA02 

Title:  GSH DualShock Wireless Controller for PlayStation 3 | Professional PS3 Wireless Gamepad for PlayStation 3/ PS3 Slim / PS3 Super Slim/PS3 Fat by - Ghost Black Limited Edition ( Only Ps3 Compitible , charging cable not included )
Price:  ₹ 909.00
link:  https://www.amazon.in/GSH-DualShock-Controller-PlayStation-Professional/dp/B08G8XDBXZ/ref=zg_bs_videogames_28/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34XVK67CGGA02 

Title:  WWE 2K20 (PS4)
Price:  ₹ 1,599.00
link:  https://www.amazon.in/2K-Games-WWE-2K20-PS4/dp/B07W6G9SCB/ref=zg_bs_videogames_34/260-6238715-2286833?_encoding=UTF8&psc=1&refRID=GMZY5GJ34XVK67CGGA02 

Title:  Marvel's Avengers (Free PS5 Upgrade)
Price:  ₹ 1,947.00
link:  https://www.amazon.in/Square-Enix-Marvels-Avengers-PS4/dp/B07YSTD

In [15]:
# https://www.amazon.in/Cosmic-Byte-GS410-Headphones-Grey/dp/B07K7XRJTZ/
# https://www.amazon.in/Sony-Uncharted-Collection-Hits-PS4/dp/B07M5V17R9/

# <base_url>/<slug>/dp/<product_id>/